In [10]:
!pip install pytorch-lightning
!pip install imports
!pip install data

  Preparing metadata (setup.py) ... done
  Created wheel for data: filename=data-0.4-py3-none-any.whl size=7227 sha256=c84c55e9fce8d9494d9e79bf57150a88375cb3a6ef003bf27f934bd1cd10c1a1
  Stored in directory: /root/.cache/pip/wheels/d0/e8/fa/e253c256048ea58d99a8abb5e751abb6a838af6f12887b5418
Successfully built data


In [5]:
#=============================#
#common imports               #
#=============================#
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import re
import numpy as np
import math
# import gradio as gr
# use this only on google colab
#from google.colab import files
from glob import glob
import os

#=============================#
#pytorch imports              #
#=============================#
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, datasets, transforms

#=============================#
#pytorch-lightning imports    #
#=============================#

import pytorch_lightning as pl
import torchmetrics as tm
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor, Callback

In [9]:
from imports import *

URL = 'https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded'
response = requests.get(URL)

store_ds_path = ''
fname = os.path.join(store_ds_path, 'data.zip')
with open(fname, 'wb') as f:
    f.write(response.content)

with zipfile.ZipFile(fname, 'r') as f:
    f.extractall()


def loader(path):
    im = cv2.imread(path)
    return im


path = os.path.join(store_ds_path,
                    'Plant_leave_diseases_dataset_without_augmentation')

tfms = transforms.Compose([
    lambda x: cv2.resize(x, (150, 150)),
    transforms.ToTensor(),
    transforms.ColorJitter(brightness=0.4, contrast=0, saturation=0, hue=0),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomAffine(
        degrees=(0, 360),
        translate=(0.1, 0.1),
        scale=(0.5, 1)
    )
])

ds = datasets.ImageFolder(path, loader=loader, transform=tfms)

def beautify_labels(label):
    bucket = re.findall(r'[^,_() ]+', label)[:3]
    bucket = list(map(lambda x: x.capitalize(), bucket))
    return ''.join(bucket)

label_names = list(map(beautify_labels, ds.classes))

label2idx = {key: val for val, key in enumerate(label_names)}
idx2label = {val: key for key, val in label2idx.items()}

train_ds, val_ds, test_ds  = torch.utils.data.random_split(ds, (45448, 5000, 5000))

In [ ]:
            from imports import *
            from data import *

            class Model(pl.LightningModule):
                def __init__(self):
                    super().__init__()
                    self.model = models.resnet18(num_classes=39)
                    self.accuracy = tm.Accuracy(task='multiclass', num_classes=39)
                    self.loss_func = nn.CrossEntropyLoss()

                def forward(self, x):
                    out = self.model(x)
                    return out

                def training_step(self, batch, batch_idx):
                    data, label = batch
                    out = self(data)
                    acc = self.accuracy(out, label)
                    loss = self.loss_func(out, label)
                    self.log('Loss/train', loss)
                    self.log('Accuracy/train', acc)
                    return loss

                def validation_step(self, batch, batch_idx):
                    data, label = batch
                    out = self(data)
                    acc = self.accuracy(out, label)
                    loss = self.loss_func(out, label)
                    self.log('Loss/val', loss)
                    self.log('Accuracy/val', acc)
                    return loss

                def predict_step(self, batch, batch_idx):
                    data, label = batch
                    out = self(data)
                    _, idx = torch.max(out, dim=1)
                    return idx

                def test_step(self, batch, batch_idx):
                    data, label = batch
                    out = self(data)
                    acc = self.accuracy(out, label)
                    loss = self.loss_func(out, label)
                    self.log('Loss/test', loss)
                    self.log('Accuracy/test', acc)
                    return loss

                def configure_optimizers(self):
                    optimizer = optim.SGD(self.parameters(), lr=3e-3, weight_decay=0) # wd = 3e-3
                    steps_per_epoch = math.ceil(len(train_ds)/64)
                    scheduler = {
                        "scheduler": optim.lr_scheduler.OneCycleLR(optimizer,
                                                                  max_lr=1e-2, epochs=10,
                                                                  steps_per_epoch=steps_per_epoch),
                        "interval": "step",
                        "frequency": 1,
                    }

                    return [optimizer], [scheduler]

                def train_dataloader(self):
                    train_dl = torch.utils.data.DataLoader(train_ds, shuffle=True, batch_size=64, num_workers=2)
                    return train_dl

                def val_dataloader(self):
                    val_dl = torch.utils.data.DataLoader(val_ds, batch_size=64, num_workers=2)
                    return val_dl

                def test_dataloader(self):
                    test_dl = torch.utils.data.DataLoader(test_ds, batch_size=64, num_workers=2)
                    return test_dl

            model = Model()

            logger = pl.loggers.TensorBoardLogger('lightning_logs', name='baseline-resnet18')
            trainer = pl.Trainer(accelerator='cpu', logger=logger,
                                max_epochs=10, profiler='simple',
                                callbacks=[ModelCheckpoint(save_weights_only=True, mode="min", monitor="Loss/val"),
                                            LearningRateMonitor("step")])


            trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | model     | ResNet             | 11.2 M
1 | accuracy  | MulticlassAccuracy | 0     
2 | loss_func | CrossEntropyLoss   | 0     
-------------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.786    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]